# Loading Data 

Para o experimentos foram escolhidos XXX movimentos que foram coletados da seguinte maneira:

Para cada postura de mão, o "subject" realizou cada movimento durante 5s por vez, com um intervalo de outros 5s para repouso, para um total de 10 repetições de cada gesto. Dessa forma, obtém-se 10 amostras de cada gesto de mão, cada uma durando 5 segundos, ou seja, 5000 pontos coletados. 

O repouso foi coletado de entre alguns dos movimentos.

Os movimentos são então separados ein:

0. gesture_0: hand resting | gesto 0: mão em repouso:
1. gesture_1: close hand   | gesto 1: mão fechada

In [ ]:
import numpy as np
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import hmmlearn

gesture_samples = 2
n_gestures = 2
file_name = 'https://raw.githubusercontent.com/raphaelrcb/TCC_UnB/raphael_dev/data/gesture_0_1.csv'

# rest_hand_series = read_csv(file_name, header=1, usecols = [0,1], index_col = 0)
rest_hand_series = []
close_hand_series = []

for i in range(gesture_samples):
    rest_hand_file =    file_name[:len(file_name)-5] + str(i+1) + file_name[len(file_name)-4:]
    close_hand_file =   file_name[:len(file_name)-7] + "1_" + str(i+1) + file_name[len(file_name)-4:]
    # print(rest_hand_file)
    rest_hand_series.append(read_csv(file_name, header=1, usecols = [0,1], index_col = 0))
    rest_hand_series[i].index = pd.to_datetime(rest_hand_series[i].index, unit = 's', origin= 'unix')
    rest_hand_series[i].index = rest_hand_series[i].index.map(lambda t: t.replace(year=2021, month=10, day=31, hour=00))
    rest_hand_series[i].index = pd.DatetimeIndex(rest_hand_series[i].index).to_period('L')
    # print(close_hand_file)
    close_hand_series.append(read_csv(file_name, header=1, usecols = [0,1], index_col = 0))
    close_hand_series[i].index = pd.to_datetime(close_hand_series[i].index, unit = 's', origin= 'unix')
    close_hand_series[i].index = close_hand_series[i].index.map(lambda t: t.replace(year=2021, month=10, day=31, hour=00))
    close_hand_series[i].index = pd.DatetimeIndex(close_hand_series[i].index).to_period('L')

# print(rest_hand_series[0].head())

# Segmenting Data

Como os dados para cada movimento foram coletados em uma única "rodada", ou seja, em uma coleta contínua para as 10 observações do gesto, é necessária uma segmentação manual para se separar apenas os 5s necessários de cada observação. Isso é feito offline através do software do delsys.

Como cada amostra de movimento dura 5s e é coletado numa freqûencia de 1000Hz, tem-se um total de 5000 pontos de dados, que podem ser divididos em janelas de 150ms com sobreposição de 50ms. Dessa forma, seguindo a fórmula

$$
N\; janelas = \frac{tamanho\: dos\,dados - tamanho\, da\, janela}{incremento\, da\,janela} + 1
$$

É possível obter um total de 98 janelas por segmento de dado.

In [ ]:
data_size = len(close_hand_series[0])
window_size = 150
window_overlap = 50

n_windows = (data_size - window_size)/window_overlap + 1

print(n_windows)

open_hand_window = []
close_hand_window = []

# for 

